In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from pathlib import Path

import tensorflow as tf
from keras._tf_keras import keras

/home/kvdomingo/mlops-play/.venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
gpus = tf.config.list_physical_devices("GPU")
tf.config.set_logical_device_configuration(
    gpus[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=16 * 1024)],
)
logical_gpus = tf.config.list_logical_devices("GPU")
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

1 Physical GPUs, 1 Logical GPUs


I0000 00:00:1766498369.590081    4053 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16384 MB memory:  -> device: 0, name: AMD Radeon RX 7900 XTX, pci bus id: 0000:03:00.0


In [3]:
RANDOM_SEED = 709

BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR / "data/muffin-vs-chihuahua"
TRAIN_DIR = DATA_DIR / "train"
TEST_DIR = DATA_DIR / "test"
CATEGORIES = ["chihuahua", "muffin"]
img_w, img_h, img_ch = IMAGE_TARGET_SIZE = (224, 224, 3)
BATCH_SIZE = 256
EPOCHS = 3

## Load data

In [4]:
train_ds: tf.data.Dataset = keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="training",
    seed=RANDOM_SEED,
    image_size=(img_h, img_w),
    batch_size=BATCH_SIZE,
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

Found 4733 files belonging to 2 classes.


Using 3787 files for training.


In [5]:
val_ds: tf.data.Dataset = keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",
    seed=RANDOM_SEED,
    image_size=(img_h, img_w),
    batch_size=BATCH_SIZE,
)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

Found 4733 files belonging to 2 classes.
Using 946 files for validation.


In [6]:
total_train = len(train_ds) * BATCH_SIZE
total_val = len(val_ds) * BATCH_SIZE

total_train, total_val

(3840, 1024)

## Model definition

In [7]:
model = keras.Sequential(
    [
        keras.layers.Input(IMAGE_TARGET_SIZE),
        keras.layers.Rescaling(1 / 255),
        keras.layers.Conv2D(32, 3, activation="relu"),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(1),
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1577088)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │   201,867,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 201,868,417 (770.07 MB)

 Trainable params: 201,868,417 (770.07 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
tensorboard = keras.callbacks.TensorBoard(
    str(BASE_DIR / ".tensorboard"),
    histogram_freq=5,
    write_images=True,
)

learning_rate = 1e-3
sgd = keras.optimizers.SGD(learning_rate)
loss = keras.losses.BinaryCrossentropy(from_logits=True)

model.compile(
    loss=loss,
    optimizer=sgd,
    metrics=["accuracy"],
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    steps_per_epoch=total_train // BATCH_SIZE,
    validation_steps=total_val // BATCH_SIZE,
    callbacks=[tensorboard],
)
results = history.history
results

Epoch 1/3


2025-12-23 22:00:08.446671: E external/local_xla/xla/stream_executor/rocm/rocm_driver.cc:1132] failed to alloc 268435456 bytes on host: HIP_ERROR_OutOfMemory
W0000 00:00:1766498408.446718    4958 device_host_allocator.h:61] could not allocate pinned host memory of size: 268435456
2025-12-23 22:00:08.568096: E external/local_xla/xla/stream_executor/rocm/rocm_driver.cc:1132] failed to alloc 241592064 bytes on host: HIP_ERROR_OutOfMemory
W0000 00:00:1766498408.568157    4958 device_host_allocator.h:61] could not allocate pinned host memory of size: 241592064
2025-12-23 22:00:08.678957: E external/local_xla/xla/stream_executor/rocm/rocm_driver.cc:1132] failed to alloc 217433088 bytes on host: HIP_ERROR_OutOfMemory
W0000 00:00:1766498408.679005    4958 device_host_allocator.h:61] could not allocate pinned host memory of size: 217433088
2025-12-23 22:00:08.777933: E external/local_xla/xla/stream_executor/rocm/rocm_driver.cc:1132] failed to alloc 195689984 bytes on host: HIP_ERROR_OutOfMemory

ResourceExhaustedError: Graph execution error:

Detected at node IteratorGetNext defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/kvdomingo/mlops-play/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/kvdomingo/mlops-play/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/kvdomingo/mlops-play/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 758, in start

  File "/home/kvdomingo/mlops-play/.venv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/home/kvdomingo/.local/share/mise/installs/python/3.12.12/lib/python3.12/asyncio/base_events.py", line 645, in run_forever

  File "/home/kvdomingo/.local/share/mise/installs/python/3.12.12/lib/python3.12/asyncio/base_events.py", line 1999, in _run_once

  File "/home/kvdomingo/.lo

2025-12-23 22:03:31.126474: E external/local_xla/xla/stream_executor/rocm/rocm_driver.cc:1132] failed to alloc 241592064 bytes on host: HIP_ERROR_OutOfMemory
W0000 00:00:1766498611.126519    4939 device_host_allocator.h:61] could not allocate pinned host memory of size: 241592064
2025-12-23 22:03:31.250703: E external/local_xla/xla/stream_executor/rocm/rocm_driver.cc:1132] failed to alloc 217433088 bytes on host: HIP_ERROR_OutOfMemory
W0000 00:00:1766498611.250755    4939 device_host_allocator.h:61] could not allocate pinned host memory of size: 217433088
2025-12-23 22:03:31.354461: E external/local_xla/xla/stream_executor/rocm/rocm_driver.cc:1132] failed to alloc 195689984 bytes on host: HIP_ERROR_OutOfMemory
W0000 00:00:1766498611.354508    4939 device_host_allocator.h:61] could not allocate pinned host memory of size: 195689984
2025-12-23 22:03:31.445498: E external/local_xla/xla/stream_executor/rocm/rocm_driver.cc:1132] failed to alloc 176121088 bytes on host: HIP_ERROR_OutOfMemory